In [1]:
# scoring as up to the day of 2018-10-01
# Track the sales from 2018-10-01 to 2019-09-30
# Need to rerun

import pandas as pd
import numpy as np
import datetime
import os
import logging
import glob
import gc
pd.options.display.max_columns=99
os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Loyalty_Health'

In [2]:
def recursive_file_gen(root_folder):
    for root, dirs, files in os.walk(root_folder):
        for file in files:
            yield os.path.join(root, file)

In [3]:
lastdate_date=datetime.date(2018,9,30) # Last day
active_Sunday=str(lastdate_date-datetime.timedelta(days=52*7-1))

lapsed_Sunday=str(lastdate_date-datetime.timedelta(days=52*7*1.5-1))
Beginning_18_months_ago=str(lastdate_date-datetime.timedelta(days=52*7*1.5-1))

lastdate=str(lastdate_date)
print("Lapsed Start on: "+lapsed_Sunday) #>=
print("Active Start on: "+active_Sunday) #>=
print("Store Allocation Starting on: "+Beginning_18_months_ago) #>=
print("Last Saturday: "+lastdate) #<=

Lapsed Start on: 2017-04-03
Active Start on: 2017-10-02
Store Allocation Starting on: 2017-04-03
Last Saturday: 2018-09-30


In [4]:
historical_subclass_files=glob.glob("/home/jian/BigLots/hist_daily_data_subclasslevel/*.txt")
historical_item_files=glob.glob("/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/*.txt")

daily_item_files_2019=list(recursive_file_gen("/home/jian/BigLots/2019_by_weeks/"))
daily_item_files_2019=[x for x in daily_item_files_2019 if x[-4:]==".txt" and "daily" in x.lower()]

daily_subclass_files_2018=list(recursive_file_gen("/home/jian/BigLots/2018_by_weeks/"))
daily_subclass_files_2018=[x for x in daily_subclass_files_2018 if x[-4:]==".txt" and "daily" in x.lower()]

historical_subclass_files.sort()
historical_item_files.sort()
daily_item_files_2019.sort()
daily_subclass_files_2018.sort()


In [5]:
# Piece 1: subclass 2017-04-16 to 2018-06-09
historical_subclass_files=[x for x in historical_subclass_files if x.split("_week_ending_")[1][:10]>=lapsed_Sunday]
'''
df=pd.read_table(historical_subclass_files[0],dtype=str,sep="|",usecols=['transaction_dt'])
print("min_date_subclass: "+str(df['transaction_dt'].min()))
del df
gc.collect()
'''
# historical subclass -- min_date_subclass: 2018-04-15


'''
df=pd.read_table(historical_subclass_files[-1],dtype=str,sep="|",usecols=['transaction_dt'])
print("max_date_subclass: "+str(df['transaction_dt'].max()))
del df
gc.collect()
'''
# historical subclass -- max_date_subclass: 2018-06-09


'\ndf=pd.read_table(historical_subclass_files[-1],dtype=str,sep="|",usecols=[\'transaction_dt\'])\nprint("max_date_subclass: "+str(df[\'transaction_dt\'].max()))\ndel df\ngc.collect()\n'

In [6]:
# Piece 2: subclass 2018-06-10 to 2018-08-04
'''
df=pd.read_table(historical_item_files[0],dtype=str,sep="|",usecols=['transaction_dt'])
print("min_date_historical_item: "+str(df['transaction_dt'].min()))
del df
gc.collect()
'''
# historical item -- min_date_historical_item: 2018-08-05
daily_subclass_files_2018=[x for x in daily_subclass_files_2018 if x.split("eks/MediaStorm_")[1][:10]>"2018-06-09"]
daily_subclass_files_2018=[x for x in daily_subclass_files_2018 if x.split("eks/MediaStorm_")[1][:10]<"2018-08-05"]



# QC the daily subclass 2018 data range
'''
df=pd.read_table(daily_subclass_files_2018[0],dtype=str,sep="|",usecols=['transaction_dt'])
print("min_date_daily_subclass: "+str(df['transaction_dt'].min()))
del df
gc.collect()
'''
# min_date_daily_subclass: 2018-06-10

'''
df=pd.read_table(daily_subclass_files_2018[-1],dtype=str,sep="|",usecols=['transaction_dt'])
print("max_date_daily_subclass: "+str(df['transaction_dt'].max()))
del df
gc.collect()
'''
# max_date_daily_subclass: 2018-08-04


'\ndf=pd.read_table(daily_subclass_files_2018[-1],dtype=str,sep="|",usecols=[\'transaction_dt\'])\nprint("max_date_daily_subclass: "+str(df[\'transaction_dt\'].max()))\ndel df\ngc.collect()\n'

In [7]:
# Piece 3: item 2018-08-05 to 2019-02-09

historical_item_files=[x for x in historical_item_files if x.split("MediaStormDailySalesHistory")[1][:8]<="20181006"]


'''
df=pd.read_table(historical_item_files[0],dtype=str,sep="|",usecols=['transaction_dt'])
print("min_date_historical_item: "+str(df['transaction_dt'].min()))
del df
gc.collect()
'''
# min_date_historical_item: 2018-08-05

'''
df=pd.read_table(historical_item_files[-1],dtype=str,sep="|",usecols=['transaction_dt'])
print("max_date_historical_item: "+str(df['transaction_dt'].max()))
del df
gc.collect()
'''
# max_date_historical_item: 2019-02-09

'\ndf=pd.read_table(historical_item_files[-1],dtype=str,sep="|",usecols=[\'transaction_dt\'])\nprint("max_date_historical_item: "+str(df[\'transaction_dt\'].max()))\ndel df\ngc.collect()\n'

In [8]:
list_all_files=historical_subclass_files+daily_subclass_files_2018+historical_item_files
print(len(list_all_files))
# list_all_files

79


In [9]:
list_df_transactions_18_months=[]
i_counter=0
for file in list_all_files:
    try:
        df=pd.read_table(file,dtype=str,sep="|",usecols=['customer_id_hashed','transaction_dt','transaction_id','location_id','item_transaction_amt'])
        df=df.rename(columns={"item_transaction_amt":"sales"})
    except:
        df=pd.read_table(file,dtype=str,sep="|",usecols=['customer_id_hashed','transaction_dt','transaction_id','location_id','subclass_transaction_amt'])
        df=df.rename(columns={"subclass_transaction_amt":"sales"})
    df=df[pd.notnull(df['customer_id_hashed'])]
    df['sales']=df['sales'].astype(float)
    df_sales=df.groupby(['customer_id_hashed','transaction_dt'])['sales'].sum().to_frame().reset_index()
    
    df_trans=df[['customer_id_hashed','transaction_dt','transaction_id','location_id']].drop_duplicates()
    df_trans['trans']=1
    df_trans=df_trans.groupby(['customer_id_hashed','transaction_dt'])['trans'].sum().to_frame().reset_index()
    df=pd.merge(df_sales,df_trans,on=["customer_id_hashed",'transaction_dt'],how="outer")

    print(i_counter,df['transaction_dt'].min(),df['transaction_dt'].max(),datetime.datetime.now(),file)
    i_counter+=1
    list_df_transactions_18_months.append(df)

0 2017-04-02 2017-04-08 2019-11-12 18:22:53.294008 /home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2017-04-08.txt
1 2017-04-09 2017-04-15 2019-11-12 18:23:36.592387 /home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2017-04-15.txt
2 2017-04-16 2017-04-22 2019-11-12 18:24:11.347530 /home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2017-04-22.txt
3 2017-04-23 2017-04-29 2019-11-12 18:24:57.201827 /home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2017-04-29.txt
4 2017-04-30 2017-05-06 2019-11-12 18:25:39.400161 /home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2017-05-06.txt
5 2017-05-07 2017-05-13 2019-11-12 18:26:21.412892 /home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2017-05-13.txt
6 2017-05-14 2017-05-20 2019-11-12 18:26:59.900644 /home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDa

56 2018-04-29 2018-05-05 2019-11-12 19:14:56.876952 /home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2018-05-05.txt
57 2018-05-06 2018-05-12 2019-11-12 19:15:46.607926 /home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2018-05-12.txt
58 2018-05-13 2018-05-19 2019-11-12 19:16:32.356598 /home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2018-05-19.txt
59 2018-05-20 2018-05-26 2019-11-12 19:17:25.218115 /home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2018-05-26.txt
60 2018-05-27 2018-06-02 2019-11-12 19:18:17.660833 /home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2018-06-02.txt
61 2018-06-03 2018-06-09 2019-11-12 19:19:07.153382 /home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2018-06-09.txt
62 2018-06-10 2018-06-16 2019-11-12 19:19:54.866082 /home/jian/BigLots/2018_by_weeks/MediaStorm_2018-06-16

In [10]:
df_transactions_18_months=pd.concat(list_df_transactions_18_months)
del list_df_transactions_18_months
gc.collect()


0

In [11]:
df_transactions_18_months=df_transactions_18_months[(df_transactions_18_months['transaction_dt']>=str(lapsed_Sunday)) & (df_transactions_18_months['transaction_dt']<=str(lastdate_date))]
df_transactions_18_months.shape

(87255633, 4)

In [12]:
df_transactions_18_months.head(2)

,customer_id_hashed,transaction_dt,sales,trans
0,000010e1fda19a1d87e283791f0abfebef1911f771f0b1...,2017-04-04,21.75,1
1,0000141dbeb91820de94321c672f0a4cf3b763e31739ce...,2017-04-08,27.25,1


In [13]:
print(df_transactions_18_months['transaction_dt'].nunique(),df_transactions_18_months['transaction_dt'].max(),df_transactions_18_months['transaction_dt'].min())


545 2018-09-30 2017-04-03


In [14]:
agg_f={"transaction_dt":"max","sales":"sum","trans":"sum"}
df_sales_by_id=df_transactions_18_months.groupby("customer_id_hashed")['transaction_dt','sales','trans'].agg(agg_f).reset_index()
df_sales_by_id=df_sales_by_id.rename(columns={"transaction_dt":"last_trans_date"})
df_sales_by_id.shape

(18871088, 4)

In [15]:
df_sales_by_id.head(2)

,customer_id_hashed,last_trans_date,sales,trans
0,000001dadc0265bf9d250566d74e0006323f18b5826641...,2018-09-22,58.70,1
1,000003066bafc6b19c55d3be5fc646a253c659e9e6baea...,2017-11-10,39.99,1


In [16]:
lastdate_date

datetime.date(2018, 9, 30)

In [17]:
df_sales_by_id['last_trans_date']=pd.to_datetime(df_sales_by_id['last_trans_date'])
df_sales_by_id['recency']= lastdate_date - df_sales_by_id['last_trans_date'].dt.date
df_sales_by_id['recency']=df_sales_by_id['recency'].apply(lambda x:x.days)
df_sales_by_id['recency']=np.ceil((df_sales_by_id['recency']+1)/30)

In [18]:
df_sales_by_id=df_sales_by_id.sort_values(['trans','recency','sales'],ascending=[0,1,0])
df_sales_by_id.reset_index(drop=True, inplace=True)
df_sales_by_id.reset_index(inplace=True)
df_sales_by_id=df_sales_by_id.rename(columns={'index':'Transindex'})

df_sales_by_id=df_sales_by_id.sort_values(['sales','recency','trans'],ascending=[0,1,0])
df_sales_by_id.reset_index(drop=True, inplace=True)
df_sales_by_id.reset_index(inplace=True)
df_sales_by_id=df_sales_by_id.rename(columns={'index':'Amtindex'})

df_sales_by_id=df_sales_by_id.sort_values(['recency','trans','sales'],ascending=[1,0,0])
df_sales_by_id.reset_index(drop=True, inplace=True)
df_sales_by_id.reset_index(inplace=True)
df_sales_by_id=df_sales_by_id.rename(columns={'index':'recencyindex'})

c_ids=len(df_sales_by_id.index)
c_ids=c_ids//5

df_sales_by_id['Transindex']=np.ceil((df_sales_by_id['Transindex']+1)/c_ids)
df_sales_by_id['Amtindex']=np.ceil((df_sales_by_id['Amtindex']+1)/c_ids)
df_sales_by_id['recencyindex']=np.ceil((df_sales_by_id['recencyindex']+1)/c_ids)

df_sales_by_id['RFM']=df_sales_by_id['recencyindex']*100 + df_sales_by_id['Transindex']*10 + df_sales_by_id['Amtindex']


In [19]:
df_sales_by_id = df_sales_by_id.sort_values(['RFM','recency','trans',
                               'sales'],ascending = [1,1,0,0])

print(df_sales_by_id.shape)

df_sales_by_id.reset_index(drop = True, inplace = True)
df_sales_by_id.reset_index(inplace = True)

df_sales_by_id = df_sales_by_id.rename(columns = {'index':'frmindex'})
c_ids = len(df_sales_by_id.index)//10+1

df_sales_by_id['frmindex'] = np.ceil((df_sales_by_id['frmindex']+1)/c_ids)

print("Done with the RFM scoring, "+str(datetime.datetime.now()))

df_sales_by_id['frmindex']=df_sales_by_id['frmindex'].apply(lambda x: "D"+str(int(float(x))).zfill(2))

(18871088, 9)
Done with the RFM scoring, 2019-11-12 20:48:45.094524


In [20]:
df_H=pd.DataFrame({"frmindex":['D01','D02','D03','D04']})
df_H['HML_Group']="H"

df_M=pd.DataFrame({"frmindex":['D05','D06','D07']})
df_M['HML_Group']="M"

df_L=pd.DataFrame({"frmindex":['D08','D09','D10']})
df_L['HML_Group']="L"

df_HML=df_H.append(df_M).append(df_L)

df_HML.head(2)

,frmindex,HML_Group
0,D01,H
1,D02,H


In [21]:
df_sales_by_id=pd.merge(df_sales_by_id,df_HML,on="frmindex",how="left")
print(df_sales_by_id.shape)

(18871088, 11)


In [22]:
df_master=pd.read_csv('/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q4/BL_all_master_id_email_zip_closet_store_ZipLabel_upto_20191012_JL_2019-10-21',
                       usecols=['customer_id_hashed','cleaned_zip_cd','zip_type'],dtype=str)

print(df_master.shape,df_master['customer_id_hashed'].nunique())

(30732606, 3) 30732606


In [23]:
df_sales_by_id=pd.merge(df_sales_by_id,df_master,on="customer_id_hashed",how="left")

In [24]:
df_sales_by_id.to_csv("./BL_Loyalty_Health_18_19_HML_score_details_JL_"+str(datetime.datetime.now().date())+".csv",index=False)

In [25]:
df_sales_by_id.head(2)

,frmindex,recencyindex,Amtindex,Transindex,customer_id_hashed,last_trans_date,sales,trans,recency,RFM,HML_Group,cleaned_zip_cd,zip_type
0,D01,1.0,1.0,1.0,2f5db30f6367de199f1a4db628fe77ebe1c6e27cd35a39...,2018-09-28,48700.51,2659,1.0,111.0,H,00nan,NaN
1,D01,1.0,1.0,1.0,dd66c64db55c9c3ce033192ad9c7ddae635a65f3d4e545...,2018-09-30,40232.94,2347,1.0,111.0,H,37311,trans_P


In [26]:
df_sales_by_id['zip_type']=df_sales_by_id['zip_type'].fillna("T")
df_sales_by_id['zip_type']=df_sales_by_id['zip_type'].replace("trans_P","P").replace("trans_S","S").replace("zips_10","S")

In [27]:
df_sales_by_id=df_sales_by_id[['customer_id_hashed','frmindex','last_trans_date','HML_Group','zip_type']]
gc.collect()

0

In [28]:
df_sales_by_id['Active_Status']=np.where(df_sales_by_id['last_trans_date']>=active_Sunday,"Active (0-12)","Lapsed (13-18)")


In [29]:
df_sales_by_id['last_trans_date'].min()

Timestamp('2017-04-03 00:00:00')

In [30]:
lastdate_date

datetime.date(2018, 9, 30)

In [31]:
Beginning_18_months_ago

'2017-04-03'

In [32]:
# Lapsed 19 - 48


Beginning_18_months_ago=str(lastdate_date-datetime.timedelta(days=52*7*1.5-1))
Beginning_48_months_ago=str(lastdate_date-datetime.timedelta(days=52*7*4-1))

lastdate=str(lastdate_date)
print("Lapsed Start on: "+lapsed_Sunday) #>=
print("Active Start on: "+active_Sunday) #>=
print("Store Allocation Starting on: "+Beginning_18_months_ago) #>=
print("Last Saturday: "+lastdate) #<=
print("Beginning_48_months_ago:"+Beginning_48_months_ago)


def recursive_file_gen(root_folder):
    for root, dirs, files in os.walk(root_folder):
        for file in files:
            yield os.path.join(root, file)



Lapsed Start on: 2017-04-03
Active Start on: 2017-10-02
Store Allocation Starting on: 2017-04-03
Last Saturday: 2018-09-30
Beginning_48_months_ago:2014-10-06


In [33]:
# 1 out of the 3

chunksize_num = 10**7
filename='/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q1/crm_newscore_0922/combinedtransactions_0922.csv'
df_lapsed_sales_ids=pd.DataFrame()
count_i=0

for chunk in pd.read_csv(filename, chunksize=chunksize_num,dtype=str,usecols=['customer_id_hashed','transaction_date']):
    chunk=chunk[(chunk['transaction_date']>=Beginning_48_months_ago) & (chunk['transaction_date']<Beginning_18_months_ago)]
    chunk=chunk.drop_duplicates()
    
    df_lapsed_sales_ids=df_lapsed_sales_ids.append(chunk)
    count_i+=1
    print(count_i,datetime.datetime.now())

del chunk

print("Earliest Date:" + str(df_lapsed_sales_ids['transaction_date'].min()))
print("Latest Date:" + str(df_lapsed_sales_ids['transaction_date'].max()))
del df_lapsed_sales_ids['transaction_date']
df_lapsed_sales_ids=df_lapsed_sales_ids.drop_duplicates()
print(df_lapsed_sales_ids.shape)
logging.info("Deduped: "+str(datetime.datetime.now()))

gc.collect()

1 2019-11-12 21:01:25.465504
2 2019-11-12 21:01:39.646691
3 2019-11-12 21:01:52.093451
4 2019-11-12 21:02:13.868441
5 2019-11-12 21:02:55.337812
6 2019-11-12 21:03:21.901323
7 2019-11-12 21:04:06.677500
8 2019-11-12 21:04:59.419332
9 2019-11-12 21:05:43.272110
10 2019-11-12 21:06:12.972574
11 2019-11-12 21:06:33.473311
12 2019-11-12 21:06:52.054357
13 2019-11-12 21:07:12.293588
14 2019-11-12 21:07:33.444118
15 2019-11-12 21:07:49.482855
Earliest Date:2016-06-26
Latest Date:2017-04-02
(12564586, 1)


0

In [34]:
# 2 out of the 3

lapsed_trans=pd.read_table("/home/jian/Projects/Big_Lots/Loyal_members/loyalty_sales_data/lapsed20140826_20170226/MediaStormLapsedCustDtl.txt",
                     sep=",",usecols=['customer_id_hashed','transaction_date'],dtype=str)# .drop_duplicates() # Doesn't go to score at all, so no need to read all columns
lapsed_trans=lapsed_trans[(lapsed_trans['transaction_date']>=Beginning_48_months_ago) & (lapsed_trans['transaction_date']<Beginning_18_months_ago)]
del lapsed_trans['transaction_date']

df_lapsed_sales_ids=df_lapsed_sales_ids.append(lapsed_trans).drop_duplicates()
del lapsed_trans
gc.collect()

0

In [35]:
# 3 out of the 3

daily_sales_historical=glob.glob("/home/jian/BigLots/hist_daily_data_subclasslevel/*.txt")
daily_sales_historical=[x for x in daily_sales_historical if x.split("MediaStormDailySales_week_ending_")[1][:10]>=Beginning_48_months_ago]
daily_sales_historical=[x for x in daily_sales_historical if x.split("MediaStormDailySales_week_ending_")[1][:10]<Beginning_18_months_ago]
print(len(daily_sales_historical))

i_counter=0
list_historical_19_48=[]
for file in daily_sales_historical:
    df=pd.read_table(file,dtype=str,sep="|",usecols=['customer_id_hashed']).drop_duplicates() # No date read, since will exclude the shoppers in 18 later
    df=df[pd.notnull(df['customer_id_hashed'])]
    list_historical_19_48.append(df)
    i_counter+=1
    if i_counter%10==1:
        print(i_counter,datetime.datetime.now())
df=pd.concat(list_historical_19_48).drop_duplicates()

df_lapsed_sales_ids=df_lapsed_sales_ids.append(df).drop_duplicates()
del df
gc.collect()


40
1 2019-11-12 21:09:42.874592
11 2019-11-12 21:12:09.529540
21 2019-11-12 21:14:18.791225
31 2019-11-12 21:16:44.735869


0

In [36]:
df_lapsed_sales_ids=df_lapsed_sales_ids[~df_lapsed_sales_ids['customer_id_hashed'].isin(df_sales_by_id['customer_id_hashed'].tolist())]
df_lapsed_sales_ids['Active_Status']="Lapsed (19-48)"
df_lapsed_sales_ids.shape

(4486353, 2)

In [37]:
df_sales_by_id=df_sales_by_id.append(df_lapsed_sales_ids)

/home/jian/.local/lib/python3.6/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [38]:
del df_lapsed_sales_ids
df_sales_by_id.to_csv("./BL_Loyalty_Health_2018_2019_all_ids_JL_"+str(datetime.datetime.now().date())+".csv",index=False)
gc.collect()

0

# Sale in 1 Year

In [39]:
historical_item_files=glob.glob("/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/*.txt")
historical_item_files=[x for x in historical_item_files if x.split("/MediaStormDailySalesHistory")[1][:8]>="201810"]
historical_item_files.sort()
historical_item_files

['/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181006.txt',
 '/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181013.txt',
 '/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181020.txt',
 '/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181027.txt',
 '/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181103.txt',
 '/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181110.txt',
 '/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181117.txt',
 '/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181124.txt',
 '/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181201.txt',
 '/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHi

In [40]:
daily_item_files_2019=[x for x in daily_item_files_2019 if x.split("eks/MediaStorm_")[1][:10]>="2019-02-16"]
daily_item_files_2019=[x for x in daily_item_files_2019 if x.split("eks/MediaStorm_")[1][:10]<="2019-10-07"]
daily_item_files_2019

['/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-02-16/MediaStormDailySales20190219-113605-481.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-02-23/MediaStormDailySales20190226-112611-940.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-03-02/MediaStormDailySales20190305-112738-899.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-03-09/MediaStormDailySales20190312-121205-996.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-03-16/MediaStormDailySales20190319-112334-057.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-03-23/MediaStormDailySales20190326-112740-903.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-03-30/MediaStormDailySales20190402-112556-551.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-04-06/MediaStormDailySales20190409-122759-093.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-04-13/MediaStormDailySales20190416-112824-908.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-04-20/MediaStormDailyS

In [41]:
list_POS_item_in_1_year=historical_item_files+daily_item_files_2019
len(list_POS_item_in_1_year)

53

In [42]:
df_sales_in_1_year=pd.DataFrame()
i_counter=0
for file in list_POS_item_in_1_year:
    try:
        df=pd.read_table(file,dtype=str,sep="|",usecols=['customer_id_hashed','transaction_dt','transaction_id','location_id','item_transaction_amt'])
        df=df.rename(columns={"item_transaction_amt":"sales"})
    except:
        df=pd.read_table(file,dtype=str,sep="|",usecols=['customer_id_hashed','transaction_dt','transaction_id','location_id','subclass_transaction_amt'])
        df=df.rename(columns={"subclass_transaction_amt":"sales"})
        
    df=df[pd.notnull(df['customer_id_hashed'])]
    ###
    df=df[df['transaction_dt']>="2018-10-01"]
    df=df[df['transaction_dt']<="2019-09-30"]
    ###
    df['sales']=df['sales'].astype(float)
    df_sales=df.groupby(['customer_id_hashed','transaction_dt'])['sales'].sum().to_frame().reset_index()
    
    df_trans=df[['customer_id_hashed','transaction_dt','transaction_id','location_id']].drop_duplicates()
    df_trans['trans']=1
    df_trans=df_trans.groupby(['customer_id_hashed','transaction_dt'])['trans'].sum().to_frame().reset_index()
    df=pd.merge(df_sales,df_trans,on=["customer_id_hashed",'transaction_dt'],how="outer")

    print(i_counter,df['transaction_dt'].min(),df['transaction_dt'].max(),datetime.datetime.now(),file)
    i_counter+=1
    df_sales_in_1_year=df_sales_in_1_year.append(df)

0 2018-10-01 2018-10-06 2019-11-12 21:27:21.361594 /home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181006.txt
1 2018-10-07 2018-10-13 2019-11-12 21:28:16.897083 /home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181013.txt
2 2018-10-14 2018-10-20 2019-11-12 21:29:12.490828 /home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181020.txt
3 2018-10-21 2018-10-27 2019-11-12 21:30:13.311513 /home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181027.txt
4 2018-10-28 2018-11-03 2019-11-12 21:31:17.752160 /home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181103.txt
5 2018-11-04 2018-11-10 2019-11-12 21:32:25.520403 /home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181110.txt
6 2018-11-11 2018-11-17 2019-11-12 21:33:45.529916 /home/jian/BigLots/hist_daily_data_itemlevel_decompress

In [43]:
gc.collect()

agg_f={"transaction_dt":"max","sales":"sum","trans":"sum"}
df_sales_in_1_year=df_sales_in_1_year.groupby("customer_id_hashed")['transaction_dt','sales','trans'].agg(agg_f).reset_index()
df_sales_in_1_year=df_sales_in_1_year.rename(columns={"transaction_dt":"last_trans_date_in_1_year"})
df_sales_in_1_year=df_sales_in_1_year.rename(columns={"sales":"sales_in_1_year"})
df_sales_in_1_year=df_sales_in_1_year.rename(columns={"trans":"trans_in_1_year"})

df_sales_in_1_year.shape

(18603131, 4)

In [44]:
df_sales_in_1_year.head(2)

,customer_id_hashed,last_trans_date_in_1_year,sales_in_1_year,trans_in_1_year
0,000000ebcf6c6a2f4302291cc9babb0760208fc683b3b5...,2019-08-29,3.80,1
1,00000135f48c68690ad3d5fc9ada41bb5cd687452007e8...,2018-10-30,211.78,3


In [45]:
df_sales_in_1_year['customer_id_hashed'].nunique()

18603131

In [46]:
df_sales_by_id.shape

(23357441, 6)

In [47]:
print(df_sales_by_id.shape)
df_sales_by_id=pd.merge(df_sales_by_id,df_sales_in_1_year,on="customer_id_hashed",how="left")
print(df_sales_by_id.shape)
df_sales_by_id['sales_in_1_year']=df_sales_by_id['sales_in_1_year'].fillna(0)
df_sales_by_id['trans_in_1_year']=df_sales_by_id['trans_in_1_year'].fillna(0)


(23357441, 6)
(23357441, 9)


In [48]:
del df_sales_in_1_year
gc.collect()

0

In [49]:
os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Loyalty_Health'

In [50]:
df_sales_by_id.to_csv("./BL_2018Audience_Performance_in_detail_19_JL_"+str(datetime.datetime.now().date())+".csv",index=False)

In [51]:
df_sales_by_id['shoppers_in_1_year']=np.where(pd.isnull(df_sales_by_id['last_trans_date_in_1_year']),0,1)
df_sales_by_id['zip_type']=df_sales_by_id['zip_type'].fillna("T")
df_sales_by_id['HML_Group']=df_sales_by_id['HML_Group'].fillna("nan")

In [52]:
df_sales_by_id['Active_Status'].unique()

array(['Active (0-12)', 'Lapsed (13-18)', 'Lapsed (19-48)'], dtype=object)

In [53]:
df_sales_by_id['HML_Group'].unique()

array(['H', 'M', 'L', 'nan'], dtype=object)

In [54]:
df_sales_by_id['zip_type'].unique()

array(['T', 'P', 'S'], dtype=object)

In [62]:
df_sales_by_id.shape,df_sales_by_id['customer_id_hashed'].nunique()

((23357441, 10), 23357441)

In [63]:
df_sales_by_id.head(2)

,Active_Status,HML_Group,customer_id_hashed,frmindex,last_trans_date,zip_type,last_trans_date_in_1_year,sales_in_1_year,trans_in_1_year,shoppers_in_1_year
0,Active (0-12),H,2f5db30f6367de199f1a4db628fe77ebe1c6e27cd35a39...,D01,2018-09-28,T,2019-05-16,877.38,53.0,1
1,Active (0-12),H,dd66c64db55c9c3ce033192ad9c7ddae635a65f3d4e545...,D01,2018-09-30,P,2018-10-15,-5.70,3.0,1


In [56]:
agg_f={"customer_id_hashed":"count","shoppers_in_1_year":"sum","sales_in_1_year":"sum","trans_in_1_year":"sum"}

df_output=df_sales_by_id.groupby(["Active_Status",'HML_Group','zip_type'])['customer_id_hashed','shoppers_in_1_year','sales_in_1_year','trans_in_1_year'].agg(agg_f).reset_index()
df_output.shape

(13, 7)

In [57]:
df_output.to_csv("./BL_2018Audience_Performance_in_summary_19_JL_"+str(datetime.datetime.now().date())+".csv",index=False)

In [58]:
df_output

,Active_Status,HML_Group,zip_type,customer_id_hashed,shoppers_in_1_year,sales_in_1_year,trans_in_1_year
0,Active (0-12),H,P,4821245,4044130,1.050063e+09,32374924.0
1,Active (0-12),H,S,1673803,1351390,3.326459e+08,9250986.0
2,Active (0-12),H,T,1053388,727156,1.589735e+08,4229824.0
3,Active (0-12),L,P,1843668,740653,8.347527e+07,1967423.0
4,Active (0-12),L,S,787297,302226,3.515459e+07,756247.0
5,Active (0-12),L,T,413151,138143,1.632324e+07,344801.0
6,Active (0-12),M,P,3464852,2158231,3.296508e+08,8183925.0
7,Active (0-12),M,S,1353427,806517,1.252245e+08,2830635.0
8,Active (0-12),M,T,843048,422896,6.323831e+07,1362926.0
9,Lapsed (13-18),L,P,1619477,520183,6.040963e+07,1347603.0


In [59]:
df_output['customer_id_hashed'].sum()

23357441

In [60]:
df_sales_by_id[(df_sales_by_id['Active_Status']=="Lapsed (13-18)") & (df_sales_by_id['HML_Group']=="nan")]

,Active_Status,HML_Group,customer_id_hashed,frmindex,last_trans_date,zip_type,last_trans_date_in_1_year,sales_in_1_year,trans_in_1_year,shoppers_in_1_year


In [61]:
print(datetime.datetime.now())

2019-11-12 23:58:39.860519
